In [ ]:
# Cell 1: Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

# Cell 2: Install necessary libraries
!pip install moviepy bm3d

# Cell 3: Import libraries
import numpy as np
import moviepy.editor as mp
from bm3d import bm3d  # Import BM3D denoising function

# Cell 4: Define function to resize frames
def resize_frame(frame, target_resolution=(1920, 1080)):
    """
    Resize a frame to the target resolution.

    Parameters:
    - frame: input frame
    - target_resolution: target resolution (width, height)

    Returns:
    - resized_frame: resized frame
    """
    resized_frame = cv2.resize(frame, target_resolution)
    return resized_frame

# Cell 5: Define function for patch-based denoising with BM3D
def patch_based_denoising(frames):
    """
    Apply patch-based denoising using BM3D to a list of frames.

    Parameters:
    - frames: list of frames

    Returns:
    - denoised_frames: list of denoised frames
    """
    # Adjust sigma_psd if needed (higher values increase denoising strength but also computation time)
    denoised_frames = [bm3d(frame, sigma_psd=2) for frame in frames]
    return denoised_frames

# Cell 6: Specify directory where the video is located
video_directory = '/content/drive/MyDrive/Colab Notebooks/video denoising media/noisy original video/'

# Cell 7: Specify the name of the input video file
video_filename = 'bike test.mp4'

# Cell 8: Process and denoise the video with patch-based denoising
video_path = video_directory + video_filename

video_clip = mp.VideoFileClip(video_path)
fps = video_clip.fps
duration = video_clip.duration
segment_duration = 7  # Process 7 seconds of video at a time (adjust as needed)
num_segments = int(np.ceil(duration / segment_duration))

output_frames_patch = []

for i in range(num_segments):
    start_time = i * segment_duration
    end_time = min((i + 1) * segment_duration, duration)
    segment_clip = video_clip.subclip(start_time, end_time)

    # Convert segment to frames
    segment_frames = [frame for frame in segment_clip.iter_frames()]

    # Apply patch-based denoising
    denoised_frames_patch = patch_based_denoising(segment_frames)
    output_frames_patch.extend(denoised_frames_patch)

# Cell 9: Reduce resolution of denoised frames
target_resolution = (480, 360)  # Adjust target resolution as needed
output_frames_patch_resized = [resize_frame(frame, target_resolution) for frame in output_frames_patch]

# Cell 10: Specify directory where the denoised video will be saved
output_directory = '/content/drive/MyDrive/Colab Notebooks/video denoising media/denoised videos/'

# Cell 11: Create a new video clip with patch-based denoised frames
denoised_video_clip_patch = mp.ImageSequenceClip(output_frames_patch_resized, fps=fps)

# Cell 12: Define output video file path for patch-based denoised video
output_video_path_patch = output_directory + 'bike_test_patch_denoised_video.mp4'

# Cell 13: Write patch-based denoised video clip to the output file
denoised_video_clip_patch.write_videofile(output_video_path_patch)

# Cell 14: Provide download link for the patch-based denoised video
print("Patch-based denoising completed. Download the patch-based denoised video:")
print(output_video_path_patch)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
